
# IRIS VQC - Blind QUantum Computing

Este notebook executa o código de Blind Quantum Computing utilizando Qiskit,  O objetivo é demonstrar como rodar experimentos quânticos de forma segura e privada, utilizando machine learning (aplicando **QNN (EstimatorQNN)** com um **classificador VQC**) com o dataset de exemplo (Iris).

Inclui:
- Pré-processamento do Iris (binário: *Setosa vs. Versicolor*)
- Construção do circuito (Feature Map + Ansatz)
- Treinamento do VQC com **ADAM**
- Envio do circuito por época
- Métricas finais e matriz de confusão

In [4]:
# Instalar dependências
!pip install qiskit qiskit-aer qiskit-machine-learning qiskit-algorithms scikit-learn matplotlib numpy

Defaulting to user installation because normal site-packages is not writeable


In [5]:
# instalar as bibliotecas necessárias

import os
import random
import numpy as np
import collections

from qiskit import QuantumCircuit
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
from qiskit.primitives import Estimator
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier
from qiskit_machine_learning.utils.loss_functions import CrossEntropyLoss
from qiskit_algorithms.optimizers import ADAM

from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import itertools

from quantumnet.components import Network, Logger


ModuleNotFoundError: No module named 'quantumnet'

# Controle de Aleatoriedade e Reprodutibilidade
O trecho abaixo define uma semente fixa (SEED = 42) e a aplica nas bibliotecas random, numpy e torch. Isso garante que os resultados aleatórios (como inicialização de pesos e embaralhamento de dados) sejam os mesmos a cada execução do código, o que é essencial para fins de reprodutibilidade. 

Além disso, o uso de torch.use_deterministic_algorithms(True) força o PyTorch a utilizar apenas algoritmos determinísticos, evitando variações internas e tornando os experimentos mais consistentes e confiáveis.

In [ ]:

SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)

# Callback: envio do circuito por época

In [ ]:

def enviar_circuito_por_epoca(circuito, epoch, num_qubits, circuit_depth):
    print(f"[Epoch {epoch+1}] Enviando circuito para a rede...")
    try:
        rede.application_layer.run_app(
            "AC_BQC",
            alice_id=6,
            bob_id=0,
            num_qubits=num_qubits,
            scenario=2,
            circuit_depth=circuit_depth,
            circuito=circuito
        )
        print(f"[Epoch {epoch+1}] Envio concluído.")
    except Exception as e:
        print(f"[Epoch {epoch+1}] Erro ao enviar circuito: {str(e)}")


# Inicializar a rede

In [ ]:

rede = Network()
rede.set_ready_topology('grade', 8, 3, 3)
Logger.activate(Logger)
print("Rede pronta.")


# Carrega o conjunto de dados Iris
- X → dados (características/features) 
- y → rótulos (classes/labels)ris (binário)

In [ ]:

iris = load_iris()
X = iris.data
y = iris.target

scaler = StandardScaler()
X = scaler.fit_transform(X)

# Reduz para duas classes (binário)

In [ ]:

X = X[y < 2]
y = y[y < 2]


# Divide os dados 
80% treino e 20% teste


In [ ]:

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=SEED
)
print("Distribuição das classes no y_train e y_test:")
print("Treino:", collections.Counter(y_train))
print("Teste:", collections.Counter(y_test))

# Cria o circuito quântico (FeatureMap + Ansatz)

In [ ]:

num_qubits = 4
feature_map = ZZFeatureMap(num_qubits)
ansatz = RealAmplitudes(num_qubits, reps=1)

qc = QuantumCircuit(num_qubits)
qc.compose(feature_map, inplace=True)
qc.compose(ansatz, inplace=True)

print("Profundidade do circuito:", qc.depth())
qc.draw(output='mpl')


# EstimatorQNN 

In [ ]:

estimator = Estimator()

qnn = EstimatorQNN(
    circuit=qc,
    input_params=feature_map.parameters,
    weight_params=ansatz.parameters,
    estimator=estimator,
    input_gradients=True
)


## 8. Treinamento

In [ ]:

print("\nTREINANDO VQC...")

vqc = NeuralNetworkClassifier(
    neural_network=qnn,
    loss=CrossEntropyLoss(),
    optimizer=ADAM(maxiter=25),
    callback=lambda weights, loss, step: enviar_circuito_por_epoca(
        circuito=feature_map.compose(ansatz.assign_parameters(weights)),
        epoch=step,
        num_qubits=num_qubits,
        circuit_depth=feature_map.compose(ansatz).depth()
    )
)

vqc.fit(X_train, y_train)
print("Treinamento concluído.")


## 9. Avaliação: métricas e matriz de confusão

In [ ]:

print("\nCALCULANDO MÉTRICAS...")
y_pred = vqc.predict(X_test)

y_test_int = y_test.astype(int)
y_pred_int = y_pred.astype(int)

accuracy = accuracy_score(y_test_int, y_pred_int)
precision = precision_score(y_test_int, y_pred_int, average='weighted', zero_division=0)
recall = recall_score(y_test_int, y_pred_int, average='weighted', zero_division=0)
f1 = f1_score(y_test_int, y_pred_int, average='weighted', zero_division=0)

print("\nMÉTRICAS FINAIS:")
print(f"Acurácia : {accuracy:.4f}")
print(f"Precisão : {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1 Score : {f1:.4f}")